In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import copy
import os
import pickle5 as pickle
%matplotlib inline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


device

device(type='cuda', index=0)

In [2]:
!pip install pickle5

'import warnings' failed; traceback:
Traceback (most recent call last):
  File "/usr/lib/python3.7/warnings.py", line 550, in <module>
    _processoptions(sys.warnoptions)
  File "/usr/lib/python3.7/warnings.py", line 208, in _processoptions
    _setoption(arg)
  File "/usr/lib/python3.7/warnings.py", line 224, in _setoption
    import re
  File "/usr/lib/python3.7/re.py", line 127, in <module>
    import functools
  File "/usr/lib/python3.7/functools.py", line 21, in <module>
    from collections import namedtuple
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 677, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 857, in get_code
  File "<frozen importlib._bootstrap_external>", line 525, in _compile_bytecode
KeyboardInterrupt
Looking in indexes: htt

In [3]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy.random as rand
import codecs
import csv
import random
import numpy as np
import numpy.linalg as LA
import fasttext
import copy
import pickle5 as pickle
import os
import torch

'''
The data can be downloaded from https://github.com/t-davidson/hate-speech-and-offensive-language. put the labeled_data.csv in this directory and run this script using python3 process_text_data.py
'''


def load_data(file_name):
    assert(os.path.exists(file_name+'.pkl'))
    with open(file_name + '.pkl', 'rb') as f:
        data = pickle.load(f)
    return data

def save_data(data, file_path):
    with open(file_path + '.pkl','wb') as f:
        pickle.dump(data,f,pickle.HIGHEST_PROTOCOL)  

class preprocess_triage_real_data:
    def __init__(self):
        pass

    def process_hate_speech_data(self, src_file, dest_file):

        with open(src_file, 'r') as f:
            f.readline()
            dict_tweet = {}
            response_list = []
            human_annotation_list = []
            while True:
                line_full = f.readline()
                if not line_full:
                    save_data({'tweets': dict_tweet, 'y': response_list, 'y_h': human_annotation_list}, dest_file)
                    return
                else:
                    if line_full.isspace():
                        print('empty')
                    else:
                        line = line_full.split(',', 7)
                        if len(line) == 7:
                            tid = line[0]
                            tweet = line[-1]
                            dict_tweet[tid] = tweet
                            y, y_h = self.get_annotations(line[1:-1])
                            response_list.append(y)
                            human_annotation_list.append(y_h)

    def get_annotations(self, list_of_arg):
        human_response = []
        for i in [1, 2, 3]:
            if int(list_of_arg[i]) > 0:
                human_response.extend([i - 1] * int(list_of_arg[i]))
        response = int(list_of_arg[-1])
        return response, human_response

    def dict_to_txt(self, tweet_dict, file_w):
        with open(file_w, 'w') as f:
            for tweet in tweet_dict.values():
                f.write(tweet)

    def map_range(self, v, l, h, l_new, h_new):
        return float(v - l) * ((h_new - l_new) / float(h - l)) + l_new

    def convert_tweet_to_vector(self, file_dict, file_vec, file_tweet):
        epsilon = 0.01
        data_dict = load_data(file_dict)
        data_vec = {}
        n_data = len(data_dict['y'])
        print(n_data)
        data_vec['y'] = np.array(data_dict['y'])
        data_vec['c'] = np.zeros((n_data,3))
        data_vec['hpred'] = np.zeros(n_data)

        for ind, human_pred, response in zip(range(n_data), data_dict['y_h'], data_vec['y']):

            h0 = float(np.sum([hpred==0 for hpred in human_pred]))
            h1 = float(np.sum([hpred==1 for hpred in human_pred]))
            h2 = float(np.sum([hpred==2 for hpred in human_pred]))
            assert(h0 + h1 + h2 == len(human_pred))
            total_votes = float(len(human_pred))
            cc = np.array([float(h0/total_votes),float(h1/total_votes),float(h2/total_votes)])
            data_vec['c'][ind] = np.array([float(h0/total_votes),float(h1/total_votes),float(h2/total_votes)])
            for i,val in enumerate(data_vec['c'][ind]):
                if val<epsilon:
                    data_vec['c'][ind][i] = epsilon
                    data_vec['c'][ind][np.argmax(data_vec['c'][ind])] -= epsilon

            c = torch.distributions.categorical.Categorical(probs = torch.tensor(cc))
            human = np.random.choice(len(human_pred))
            #human = c.sample().item()
            data_vec['hpred'][ind] = human_pred[human]

        self.dict_to_txt(data_dict['tweets'],file_tweet)
        model = fasttext.train_unsupervised(file_tweet, model='skipgram')
        x = []
        for tid in data_dict['tweets'].keys():
            tweet = data_dict['tweets'][tid].replace('\n', ' ')
            x.append(model.get_sentence_vector(tweet).flatten())
        data_vec['x'] = np.array(x)

        save_data(data_vec, file_vec)

    def truncate_data(self, data_file, data_file_tr):
        data = load_data(data_file)
        n = data['y'].shape[0]
        n_tr = int(n / 4)
        print('x', data['x'].shape)
        print('y', data['y'].shape)
        print('c', data['c'].shape)
        data['x'] = data['x'][:n_tr]
        data['y'] = data['y'][:n_tr]
        data['c'] = data['c'][:n_tr]
        data['hpred'] = data['hpred'][:n_tr]
        print('x', data['x'].shape)
        print('y', data['y'].shape)
        print('c', data['c'].shape)
        save_data(data, data_file_tr)

    def split_data(self, frac, file_data, file_data_split):

        data = load_data(file_data)

        print('x', data['x'].shape)
        print('y', data['y'].shape)
        print('c', data['c'].shape)
        num_data = data['y'].shape[0]
        print(num_data)
        num_train = int(frac * num_data)
        num_test = int((num_data - num_train)/2)
        num_val = num_data - (num_test + num_train)
        indices = np.arange(num_data)
        random.shuffle(indices)
        indices_train = indices[:num_train]
        indices_val = indices[num_train:num_train+num_val]
        indices_test = indices[num_train+num_val:num_train+num_val+num_test]
        data_split = {}
        data_split['X'] = data['x'][indices_train]
        data_split['Y'] = data['y'][indices_train]
        data_split['c'] = data['c'][indices_train]
        data_split['hpred'] = data['hpred'][indices_train]

        val = {}
        val['X'] = data['x'][indices_val]
        val['Y'] = data['y'][indices_val]
        val['c'] = data['c'][indices_val]
        val['hpred'] = data['hpred'][indices_val]
        data_split['val'] = val

        test = {}
        test['X'] = data['x'][indices_test]
        test['Y'] = data['y'][indices_test]
        test['c'] = data['c'][indices_test]
        test['hpred'] = data['hpred'][indices_test]
        data_split['test'] = test
        data_split['dist_mat'] = np.zeros((num_test, num_train))
        save_data(data_split, file_data_split)

    def change_format_hatespeech(self, data_file, dest_file):
        data = load_data(data_file)

        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()

        x_train = scaler.fit_transform(data['X'])
        x_val = scaler.transform(data['val']['X'])
        x_test = scaler.transform(data['test']['X'])
        data['X'] = x_train
        data['val']['X'] = x_val
        data['test']['X'] = x_test

        c = {'0.0': np.copy(data['c'])}
        val_c = {'0.0': np.copy(data['val']['c'])}
        test_c = {'0.0': np.copy(data['test']['c'])}
        data['c'] = c
        data['val']['c'] = val_c
        data['test']['c'] = test_c
        save_data(data, dest_file)

def find_human_loss(file_path):
    data = load_data(file_path)
    loss_func = torch.nn.NLLLoss(reduction='none')

    Y = torch.from_numpy(copy.deepcopy(data['Y'])).long()
    hprob = torch.log2(torch.from_numpy(copy.deepcopy(data['c']['0.0'])).float())
    hloss = loss_func(hprob, Y)
    human_prob, _ = torch.max(hprob, axis=1)

    val_Y = torch.from_numpy(copy.deepcopy(data['val']['Y'])).long()
    val_hprob = torch.log2(torch.from_numpy(copy.deepcopy(data['val']['c']['0.0'])).float())
    val_hloss = loss_func(val_hprob, val_Y)
    val_human_prob, _ = torch.max(val_hprob, axis=1)


    test_Y = torch.from_numpy(copy.deepcopy(data['test']['Y'])).long()
    test_hprob = torch.log2(torch.from_numpy(copy.deepcopy(data['test']['c']['0.0'])).float())
    test_hloss = loss_func(test_hprob, test_Y)
    test_human_prob, _ = torch.max(test_hprob, axis=1)

    data['hloss'] = hloss
    data['val']['hloss'] = val_hloss
    data['test']['hloss'] = test_hloss

    data['hprob'] = human_prob
    data['val']['hprob'] = val_human_prob
    data['test']['hprob'] = test_human_prob
    save_data(data,file_path)

def main(path):
    src_file = path + 'labeled_data.csv'
    obj = preprocess_triage_real_data()
    dest_file = path + 'data'
    tweet_file = path + 'tweets.txt'
    vec_file = path + 'data_vectorized'
    vec_full_split_file = path + 'input_full'

    obj.process_hate_speech_data(src_file,dest_file)
    obj.convert_tweet_to_vector(dest_file,vec_file,tweet_file)
    obj.split_data(0.6, vec_file , vec_full_split_file)

    dest_file = path + 'hatespeech_data'
    obj.change_format_hatespeech(vec_full_split_file, dest_file)
    find_human_loss(dest_file)

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
path =  "/content/drive/MyDrive/Colab Notebooks/differentiable-learning-under-triage-main/Hatespeech/"

In [7]:
# main("/content/drive/MyDrive/Colab Notebooks/differentiable-learning-under-triage-main/Hatespeech/")

In [8]:
def load_data(file_name):
    assert(os.path.exists(file_name+'.pkl'))
    with open(file_name + '.pkl', 'rb') as f:
        data = pickle.load(f)
    return data

In [9]:
def save_data(data, file_path):
    with open(file_path + '.pkl','wb') as f:
        pickle.dump(data,f,pickle.HIGHEST_PROTOCOL)

In [10]:
import random
def set_seed(seed):
		random.seed(seed)
		np.random.seed(seed)
		torch.manual_seed(seed)
		if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)

#### The hatespeech data can be found [here](https://github.com/t-davidson/hate-speech-and-offensive-language). Features are extracted from each tweet using [fasttext](https://pypi.org/project/fasttext/). Data is preprocessed in "preprocess_text_data.py" and saved in "hatespeech_data.pkl". Refer to section 6 of the paper for a detailed description of preprocessing and human predictions modeling. In this notebook we only load the data which we previously generated using the "preprocess_text_data.py".

In [11]:
constraints = [0.2,0.4,0.6,0.8]
data_path = '/content/drive/MyDrive/Colab Notebooks/differentiable-learning-under-triage-main/Hatespeech/hatespeech_data'
model_dir = '/content/drive/MyDrive/Colab Notebooks/differentiable-learning-under-triage-main/Hatespeech/models/'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

#### The CNN architecture that we used for text classification is that of [Kim 2014](https://arxiv.org/abs/1408.5882).

In [12]:
class CNN(nn.Module):
    def __init__(self, vocab_size, n_filters, filter_sizes, output_dim,
                 dropout):
        super().__init__()
        self.conv_0 = nn.Conv1d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[0]))

        self.conv_1 = nn.Conv1d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[1]))

        self.conv_2 = nn.Conv1d(in_channels=1,
                                out_channels=n_filters,
                                kernel_size=(filter_sizes[2]))

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

        self.logsoftmax = nn.LogSoftmax(dim=-1)

    def forward(self, text):

        embedded = text.unsqueeze(1)

        conved_0 = F.relu(self.conv_0(embedded).squeeze(2))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(2))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(2))


        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)

        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        cat = self.fc(cat)
        cat = self.logsoftmax(cat).squeeze()
        return cat

#### The CNN architecture for the [surrogate-based triage](https://arxiv.org/abs/2006.01862)

In [13]:
class CNN_rej(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout):
        
        super().__init__()
                        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = fs) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
                
        self.convs_rej = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = fs) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc_rej = nn.Linear(len(filter_sizes) * n_filters, 1)
        
        self.dropout_rej = nn.Dropout(dropout)
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, embedded):
        
        embedded = embedded.unsqueeze(1)
        
        conved = [F.relu(conv(embedded)).squeeze(2) for conv in self.convs]
            
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        embedded_rej = embedded.unsqueeze(1)
                
        conved_rej = [F.relu(conv(embedded)).squeeze(2) for conv in self.convs_rej]
                            
        pooled_rej = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
                
        cat_rej = self.dropout_rej(torch.cat(pooled, dim = 1))

        out_rej = self.fc_rej(cat_rej)
        
        out = self.fc(cat)
        out =  torch.cat((out, out_rej), 1)

        out = self.softmax(out)
        return out

In [14]:
class CNN_rej_ova(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout):
        
        super().__init__()
                        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = fs) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
                
        self.convs_rej = nn.ModuleList([
                                    nn.Conv1d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = fs) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc_rej = nn.Linear(len(filter_sizes) * n_filters, 1)
        
        self.dropout_rej = nn.Dropout(dropout)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, embedded):
        
        embedded = embedded.unsqueeze(1)
        
        conved = [F.relu(conv(embedded)).squeeze(2) for conv in self.convs]
            
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        
        embedded_rej = embedded.unsqueeze(1)
                
        conved_rej = [F.relu(conv(embedded)).squeeze(2) for conv in self.convs_rej]
                            
        pooled_rej = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
                
        cat_rej = self.dropout_rej(torch.cat(pooled, dim = 1))

        out_rej = self.fc_rej(cat_rej)
        
        out = self.fc(cat)
        out =  torch.cat((out, out_rej), 1)

        conf = self.sigmoid(out)
        return out, conf

# Our Method :

#### According to the optimal triage policy, the machine only is trained on the points on which the difference between the machine loss and human loss is less than $t_{P,b,m}$ (refer to equation 3 in the paper). This is equivalent to sorting the samples based on the difference of machine loss and human loss and then giving to machine the first $\max(\lceil (1-b) \, |D| \rceil, p)$ samples where $p$ is the number of samples with $\ell(m_{\theta_{t-1}}(x_{[i]}), y_{[i]}) - \ell(h_{[i]}, y_{[i]}) < 0$.

In [15]:
def find_machine_samples(machine_loss, hloss,constraint):
    
    diff = machine_loss - hloss
    argsorted_diff = torch.clone(torch.argsort(diff))
    num_outsource = int(constraint * machine_loss.shape[0])
    index = -num_outsource

    while (index < -1 and diff[argsorted_diff[index]] <= 0):
        index += 1

    machine_list = argsorted_diff[:index]

    return machine_list

#### Here we train the machine model on its points. In each iteration first, the machine points are found and then the machine model is trained on them. We use early stopping with a patience of 10 epochs based on the performance on the validation set to determine when to stop training.

In [16]:
def train_triage(seed, data_path,constraint):  
    machine_type = 'Differentiable'
    if seed != '':
      mct = machine_type + '_seed_' + str(seed) + '_'
    else:
      mct = machine_type + '_'
    print('training machine model using constraint:',constraint,' and machine model: ',machine_type)
    data = load_data(data_path)
    X = torch.from_numpy(data['X']).float().to(device)
    Y = torch.from_numpy(data['Y']).long().to(device)
    hloss = data['hloss'].to(device)
    
    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).long().to(device)
    val_hloss = data['val']['hloss'].to(device)
    
    batch_size = 64
    num_batches = int(X.shape[0] / batch_size)
    
    num_epochs = 30
        
    mnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)

    if constraint == 0.2:
        lr = 0.005
    else:
        lr = 0.0002

    optimizer = torch.optim.Adam(mnet.parameters(),lr = lr)
    loss_func = torch.nn.NLLLoss(reduction='none')
    train_losses = []
    val_losses = []
    best_val_loss = 1000
    eps = 1e-4
    max_patience = 10
    patience = 0
    
    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        train_loss = 0
        with torch.no_grad():
            mprim = copy.deepcopy(mnet)
        machine_loss = []
        for i in range(num_batches):
            X_batch = X[i * batch_size: (i + 1) * batch_size]
            Y_batch = Y[i * batch_size: (i + 1) * batch_size]
            hloss_batch = hloss[i * batch_size: (i + 1) * batch_size]
            machine_scores_batch = mprim(X_batch)
            machine_loss_batch = loss_func(machine_scores_batch,Y_batch)
            machine_loss.extend(machine_loss_batch.detach())
            machine_indices = find_machine_samples(machine_loss_batch, hloss_batch, constraint)

            X_machine = X_batch[machine_indices]
            Y_machine = Y_batch[machine_indices]
            optimizer.zero_grad()
            loss = loss_func(mnet(X_machine),Y_machine)
            loss.sum().backward()
            optimizer.step()
            train_loss += float(loss.mean())

        train_losses.append(train_loss / num_batches)
        print('machine_loss:', train_loss/num_batches)
        
        with torch.no_grad():
            val_loss = 0
            val_machine_scores = mprim(val_X)
            val_machine_loss = loss_func(val_machine_scores,val_Y)
            
            val_machine_indices = find_machine_samples(val_machine_loss,val_hloss,constraint)
            val_loss = float(loss_func(mnet(val_X[val_machine_indices]),val_Y[val_machine_indices]).mean())
            print('val_loss:',val_loss) 
            
            if val_loss + eps < best_val_loss:
                torch.save(mnet.state_dict(), model_dir + 'm_' + mct + str(constraint))
                best_val_loss = val_loss
                print('updated the model')
                patience = 0
            else:
                patience += 1
            val_losses.append(val_loss)
        
        if patience > max_patience:
            print('no progress for 10 epochs... stopping training')
            break

        print('\n')
            
        if epoch%5==0:
            fig, ax = plt.subplots()
            plt.title('b = ' + str(constraint) + ' epoch : ' + str(epoch),fontsize=22)
            plt.xlim(-5.5,1.3)
            plt.ylim([-24,8])
            plt.xticks([-5,-2,1],[r'$2^{-5}$',r'$2^{-2}$',r'$2^0$'])
            plt.yticks([-20,-10,1],[r'$2^{-20}$',r'$2^{-10}$',r'$2^{0}$'])
            plt.xlabel(r'Human Loss',fontsize=22)
            plt.ylabel(r'Machine Loss',fontsize=22)
            line = np.linspace(-6.2,1.3,100)
            plt.plot(line,line,'--')
            machine_loss = torch.tensor(machine_loss, device = 'cpu')
            plt.scatter(np.log2(hloss[:len(machine_loss)].cpu().data.numpy()),np.log2(np.array(machine_loss)),alpha=0.3)  
            plt.show()
        
    
        
    plt.plot(range(len(train_losses)),train_losses,marker='o',label='train')
    plt.plot(range(len(val_losses)),val_losses,marker='o',label='validation')
    plt.legend()
    plt.title(machine_type + ' b = ' + str(constraint),fontsize=22)
    plt.xlabel(r'Time Step t',fontsize=22)
    plt.ylabel(r'Machine Loss',fontsize=20)
    plt.show()


In [18]:
for seed in ['', 948,  625,  436,  791, 1750]:
  print("seed: ", seed)
  if seed != '':
    set_seed(seed)
  for constraint in constraints:
      train_triage(seed, data_path,constraint)
      

#### We train the additional model g to approximate difference of machine loss and human loss for unseen samples. The same CNN and the same training procedure is used for training g. Please refer to 

In [ ]:
def train_g(seed, data_path, machine_type,constraint):
    if seed != '':
      mct = machine_type + '_seed_' + str(seed) + '_'
    else:
      mct = machine_type + '_'
    print('started training g using the constraint: ',constraint,' Using machine model: ',machine_type)
    data = load_data(data_path)
    X = torch.from_numpy(copy.deepcopy(data['X'])).float().to(device)
    Y = torch.from_numpy(copy.deepcopy(data['Y'])).long().to(device)
    hloss = data['hloss'].to(device)
        
    mnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)
    
    if machine_type == 'full':
        mnet.load_state_dict(torch.load(model_dir + 'm_full'))
        
    else:
        mnet.load_state_dict(torch.load(model_dir + 'm_' + mct + str(constraint)))
        
    mnet.eval()
    
    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).long().to(device)
    val_hloss = data['val']['hloss'].to(device)
    
    batch_size = 64
    num_batches = int(X.shape[0] / batch_size)
    
    num_epochs = 50
    
    gnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=2, dropout=0.5).to(device)
    
    g_optimizer = torch.optim.Adam(gnet.parameters(),lr=0.008)
    loss_func = torch.nn.NLLLoss(reduction='none')
    

    train_losses = []
    val_losses = []
    best_val_loss = 1000
    max_patience = 10
    patience = 0
    eps = 1e-4
    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        g_train_loss = 0
        for i in range(num_batches):
            X_batch = X[i * batch_size: (i + 1) * batch_size]
            Y_batch = Y[i * batch_size: (i + 1) * batch_size]
            hloss_batch = hloss[i * batch_size: (i + 1) * batch_size]
            machine_loss_batch = loss_func(mnet(X_batch),Y_batch)
            machine_indices = find_machine_samples(machine_loss_batch, hloss_batch, constraint)
            g_labels = torch.tensor([0 if j in machine_indices else 1 for j in range(hloss_batch.shape[0])]).to(device)
            g_optimizer.zero_grad()
            g_loss = loss_func(gnet(X_batch),g_labels)
            g_loss.sum().backward()
            g_optimizer.step()
            g_train_loss += float(g_loss.mean())
        train_losses.append(g_train_loss/num_batches)
        print('g_loss:',g_train_loss/num_batches) 
        
        with torch.no_grad():
            val_machine_loss = loss_func(mnet(val_X),val_Y)
            val_machine_indices = find_machine_samples(val_machine_loss,val_hloss,constraint)
            val_glabels = torch.tensor([0 if j in val_machine_indices else 1 for j in range(val_X.shape[0])]).to(device)
            val_loss = loss_func(gnet(val_X),val_glabels)
            val_gloss = float(val_loss.mean())
            val_losses.append(val_gloss)
            print('val_g_loss:',float(val_gloss))
            
            if val_gloss + eps < best_val_loss:
                torch.save(gnet.state_dict(), model_dir + 'g_' + mct + str(constraint))
                best_val_loss = val_gloss
                print('updated the model')
                patience = 0
            else:
                patience += 1
            
        if patience > max_patience:
            print('no progress for 10 epochs... stopping training')
            break

                
        print('\n')
        
    plt.plot(range(len(train_losses)),train_losses,marker = 'o',label='train')
    plt.plot(range(len(val_losses)),val_losses,marker = 'o',label='validation')
    plt.title('train and validation curve of g using b = ' + str(constraint),fontsize=22)
    plt.xlabel('Epoch',fontsize=22)
    plt.ylabel(r'g Loss',fontsize=20)
    plt.legend()
    plt.show()
        

In [ ]:
for seed in ['', 948,  625,  436,  791, 1750]:
  if seed != '':
    set_seed(seed)
  for constraint in constraints:
      train_g(seed, data_path,'Differentiable',constraint)

In [ ]:
def get_test_assignments_us(data_path,constraints):
    l = []
    for seed in ['', 948,  625,  436,  791, 1750]:
      machine_type = 'Differentiable'
      if seed != '':
        mct = machine_type + '_seed_' + str(seed) + '_'
      else:
        mct = machine_type + '_'
      loss_func = torch.nn.NLLLoss(reduction='none')
      data = load_data(data_path)
      test_X = torch.from_numpy(data['test']['X']).float().to(device)
      test_Y = data['test']['Y']
      hlabel = data['test']['hpred']
      
      losses = []
      for constraint in constraints:
          loss = np.zeros(test_X.shape[0])
          num_machine = int((1.0 - constraint) * test_X.shape[0])
          mnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=3, dropout=0.5).to(device)

          mnet.load_state_dict(torch.load(model_dir + 'm_' + mct + str(constraint)))
          mnet.eval()
          
          mlabel = torch.argmax(mnet(test_X),dim=1).cpu().data.numpy()
          
          gnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=2, dropout=0.5).to(device)

          gnet.load_state_dict(torch.load(model_dir + 'g_' + mct + str(constraint)))
          gnet.eval()
          
          gprediction = torch.exp(gnet(test_X).detach()[:,1])
          to_machine = torch.argsort(gprediction)[:num_machine].cpu().data.numpy()
          to_human = np.array([i for i in range(test_X.shape[0]) if i not in to_machine])
          
          mloss = np.not_equal(mlabel,test_Y)
          hloss = np.not_equal(hlabel,test_Y)
          
          loss[to_machine] = mloss[to_machine]
          loss[to_human] = hloss[to_human]

          losses.append(np.mean(loss))
          del mnet
          del gnet
      l.append(losses)
      
    if machine_type not in data.keys():
        data[machine_type] = {}
    l = np.array(l)
    print("l shape: ", l.shape)
    data[machine_type]['agg_loss'] = l
    save_data(data,data_path)

In [ ]:
get_test_assignments_us(data_path,constraints)

In [ ]:
# def boxplot():
#     constraints = [0.4, 0.6, 0.8, 1.0]
#     res = load_data('results/Differentiable')
#     for constraint in constraints:
#         fig, ax = plt.subplots()

#         gpred = res[constraint]['gprediction'].cpu()
#         mloss = res[constraint]['mcrossloss'].cpu()
#         hloss = res[constraint]['hcrossloss'].cpu()
#         to_machine = res[constraint]['to_machine']
#         to_human = res[constraint]['to_human']
#         threshold_map = {0.0: 0.0, 0.2: 0.257, 0.4: 0.257, 0.6: 0.211, 0.8: 0.211, 1.0: 0.373}
#         below = [idx for idx in range(gpred.shape[0]) if idx in to_machine]
#         above = [idx for idx in range(gpred.shape[0]) if idx in to_human]
#         below_data = [np.log2(mloss[idx]) - np.log2(hloss[idx]) for idx in below]
#         above_data = [np.log2(mloss[idx]) - np.log2(hloss[idx]) for idx in above]
        
#         medianprops = dict(linestyle='-.', linewidth=2.5, color='black')

#         ax1 = ax.boxplot(np.array(below_data), positions=[0],  widths=0.35,sym='',whis=0,medianprops=medianprops,#usermedians=[np.mean(below_data)],
#                          patch_artist=True, boxprops=dict(facecolor="C0"))
#         ax2 = ax.boxplot(np.array(above_data), positions=[1],  widths=0.35,sym='',whis=0,medianprops = medianprops,#usermedians=[np.mean(above_data)],
#                          patch_artist=True, boxprops=dict(facecolor="C1"))
#         plt.show()

In [ ]:
# boxplot()

# Baselines

## [Confidence Based Triage](https://arxiv.org/abs/2004.13102)

In [ ]:
def train_confidence(seed, data_path,constraint):
    machine_type = 'confidence'
    if seed != '':
      mct = machine_type + '_seed_' + str(seed) + '_'
    else:
      mct = machine_type + '_'
    print('-----training machine model using constraint:',constraint,' and machine model: ',machine_type)
    data = load_data(data_path)
    X = torch.from_numpy(data['X']).float().to(device)
    Y = torch.from_numpy(data['Y']).long().to(device)
    hconf = torch.mean(data['hprob']) + torch.zeros(X.shape[0]).to(device)
    
    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).long().to(device)
    val_hconf = torch.mean(data['hprob']) + torch.zeros(val_X.shape[0]).to(device)
    
    batch_size = 64
    num_batches = int(X.shape[0] / batch_size)
    
    num_epochs = 30
        
    mnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)
    
        
    optimizer = torch.optim.Adam(mnet.parameters(),lr = 0.0065)
    loss_func = torch.nn.NLLLoss(reduction='none')
    train_losses = []
    val_losses = []
    best_val_loss = 1000
    max_patience = 10
    patience = 0
    eps = 1e-4
    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        train_loss = 0
        with torch.no_grad():
            mprim = copy.deepcopy(mnet)
        machine_loss = []
        for i in range(num_batches):
            X_batch = X[i * batch_size: (i + 1) * batch_size]
            Y_batch = Y[i * batch_size: (i + 1) * batch_size]
            hconf_batch = hconf[i * batch_size: (i + 1) * batch_size]
            machine_scores_batch = mprim(X_batch)
            machine_conf_batch, _ = torch.max(machine_scores_batch,axis = 1)   
            machine_indices = find_machine_samples(hconf_batch,machine_conf_batch,constraint)
                
            X_machine = X_batch[machine_indices]
            Y_machine = Y_batch[machine_indices]
            optimizer.zero_grad()
            loss = loss_func(mnet(X_machine),Y_machine)
            loss.sum().backward()
            optimizer.step()
            train_loss += float(loss.mean())

        train_losses.append(train_loss / num_batches)
        print('machine_loss:', train_loss/num_batches)
        
        with torch.no_grad():
            val_loss = 0
            val_machine_scores = mprim(val_X)
            val_machine_conf,_ = torch.max(val_machine_scores,axis=1)

            val_machine_indices = find_machine_samples(val_hconf,val_machine_conf,constraint)

            val_loss = float(loss_func(mnet(val_X[val_machine_indices]),val_Y[val_machine_indices]).mean())
            
            val_losses.append(val_loss)
            print('val_loss:',val_loss) 
            
            if val_loss + eps <best_val_loss:
                torch.save(mnet.state_dict(), model_dir + 'm_' + mct + str(constraint))
                best_val_loss = val_loss
                print('updated the model')
                patience = 0
            else:
                patience += 1
                
        if patience > max_patience:
            print('no progress for 10 epochs... stopping training')
            break
      
        print('\n')
            
    # plt.plot(range(len(train_losses)),train_losses,marker='o',label = 'train')
    # plt.plot(range(len(val_losses)),val_losses,marker='o',label = 'validation')
    # plt.legend()
    # plt.title(machine_type + ' b = ' + str(constraint),fontsize=22)
    # plt.xlabel(r'Time Step t',fontsize=22)
    # plt.ylabel(r'Machine Loss',fontsize=20)
    # plt.show()
        

In [ ]:
for seed in ['', 948,  625,  436,  791, 1750]:
  if seed != '':
    set_seed(seed)
  for constraint in constraints:
    train_confidence(seed, data_path,constraint)

In [ ]:
def get_test_assignments_confidence(data_path,constraints):
    machine_type = 'confidence'
    l = []
    for seed in ['', 948,  625,  436,  791, 1750]:
      if seed != '':
        mct = machine_type + '_seed_' + str(seed) + '_'
      else:
        mct = machine_type + '_'

      loss_func = torch.nn.NLLLoss(reduction='none')
      data = load_data(data_path)
      test_X = torch.from_numpy(data['test']['X']).float().to(device)
      test_Y = data['test']['Y']
      hlabel = data['test']['hpred']
      hconf = (torch.mean(data['hprob']) + torch.zeros(test_X.shape[0])).to(device)
      
      losses = []
      for constraint in constraints:
          mnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=3, dropout=0.5).to(device)

          mnet.load_state_dict(torch.load(model_dir + 'm_' + mct + str(constraint)))
          mnet.eval()
          mlabel = torch.argmax(mnet(test_X),dim=1).cpu().data.numpy()
          loss = np.zeros(test_X.shape[0])
          mconf,_ = torch.max(mnet(test_X),axis = 1)
          to_machine = find_machine_samples(hconf,mconf,constraint).cpu().data.numpy()

          to_human = np.array([i for i in range(test_X.shape[0]) if i not in to_machine])

          mloss = np.not_equal(mlabel,test_Y)
          hloss = np.not_equal(hlabel,test_Y)

          loss[to_machine] = mloss[to_machine]
          loss[to_human] =  hloss[to_human]

          losses.append(np.mean(loss))
          del mnet
          
      l.append(losses)
    
    if machine_type not in data.keys():
        data[machine_type] = {}

    l = np.array(l)
    print('l shape: ', l.shape)
    data[machine_type]['agg_loss'] = l
    
    save_data(data,data_path)

In [ ]:
get_test_assignments_confidence(data_path,constraints)

## Full Automation Triage
The machine model is trained under full automation and then an additional model g is trained based on the difference of machine and human loss and used to outsource samples at test time.

In [ ]:
def train_full(seed, data_path, machine_type):
    print('-----training machine model: ',machine_type)
    if seed != '':
      mct = machine_type + '_seed_' + str(seed)
    else:
      mct = machine_type 
    data = load_data(data_path)
    X = torch.from_numpy(data['X']).float().to(device)
    Y = torch.from_numpy(data['Y']).long().to(device)
    hloss = data['hloss'].to(device)
    hconf = torch.mean(data['hprob']) + torch.zeros(X.shape[0]).to(device)
    
    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).long().to(device)
    val_hloss = data['val']['hloss'].to(device)
    val_hconf = torch.mean(data['hprob']) + torch.zeros(val_X.shape[0]).to(device)
    
    batch_size = 64
    num_batches = int(X.shape[0] / batch_size)
    val_num_batches = int(val_X.shape[0]/batch_size)
    
    num_epochs = 30
        
    mnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)

    optimizer = torch.optim.Adam(mnet.parameters(),lr=0.005)
    loss_func = torch.nn.NLLLoss(reduction='none')
    train_losses = []
    val_losses = []
    best_val_loss = 1000
    max_patience = 10
    patience = 0
    eps = 1e-4
    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        train_loss = 0
        machine_loss = []
        for i in range(num_batches):
            X_batch = X[i * batch_size: (i + 1) * batch_size]
            Y_batch = Y[i * batch_size: (i + 1) * batch_size]
            optimizer.zero_grad()
            loss = loss_func(mnet(X_batch),Y_batch)
            loss.sum().backward()
            optimizer.step()
            train_loss += float(loss.mean())

        train_losses.append(train_loss / num_batches)
        print('machine_loss:', train_loss/num_batches)
        
        with torch.no_grad():
            val_loss = float(loss_func(mnet(val_X),val_Y).mean())
            print('val_loss:',val_loss)
            val_losses.append(val_loss)
            if val_loss + eps < best_val_loss:
                torch.save(mnet.state_dict(), model_dir + 'm_' + mct)
                best_val_loss = val_loss
                print('updated the model')
                patience = 0
            else:
                patience += 1
                
        if patience > max_patience:
            print('no progress for 10 epochs... stopping training')
            break

        print('\n')
    
    
    # plt.plot(range(len(train_losses)),train_losses,marker='o',label='train')
    # plt.plot(range(len(val_losses)),val_losses,marker='o',label = 'validation')
    # plt.legend()
    # plt.title(machine_type, fontsize=22)
    # plt.xlabel(r'Time Step t',fontsize=22)
    # plt.ylabel(r'Machine Loss',fontsize=20)
    # plt.show()
        

In [ ]:
# train_full(data_path,'full')

In [ ]:
# def train_g_full(data_path):
#     machine_type = 'full'
#     print('started training g Using machine model: ',machine_type)
#     data = load_data(data_path)
#     X = torch.from_numpy(data['X']).float()
#     Y = torch.from_numpy(data['Y']).long()
#     hloss = data['hloss']
    
#     with torch.no_grad():
#         mnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=3, dropout=0.5).to(device)

#         mnet.load_state_dict(torch.load(model_dir + 'm_full'))

#         mnet.to(device)
#         mnet.eval()
    
    
#     val_X = torch.from_numpy(data['val']['X']).float()
#     val_Y = torch.from_numpy(data['val']['Y']).long()
#     val_hloss = data['val']['hloss']
    
#     batch_size = 128
#     num_batches = int(X.shape[0] / batch_size)
#     val_num_batches = int(val_X.shape[0] / batch_size)
    
#     num_epochs = 30
    
#     gnet = CNN(vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=2, dropout=0.5).to(device)
#     gnet.to(device)
#     g_optimizer = torch.optim.Adam(gnet.parameters(),lr=0.001)
#     loss_func = torch.nn.NLLLoss(reduction='none')
    

#     train_losses = []
#     val_losses = []
#     best_val_loss = 1000
#     max_patience = 8
#     patience = 0
#     eps = 1e-3
    
#     for epoch in range(num_epochs):
#         gprediction = []
#         print('----- epoch:',epoch, '-----')
#         g_train_loss = 0
#         for i in range(num_batches):
#             X_batch = X[i * batch_size: (i + 1) * batch_size].to(device)
#             Y_batch = Y[i * batch_size: (i + 1) * batch_size].to(device)
#             hloss_batch = hloss[i * batch_size: (i + 1) * batch_size].to(device)
#             with torch.no_grad():
#                 machine_loss_batch = loss_func(mnet(X_batch),Y_batch)
#                 machine_indices = find_machine_samples(machine_loss_batch, hloss_batch, constraint=1.0)
#             g_labels = torch.tensor([0 if j in machine_indices else 1 for j in range(hloss_batch.shape[0])]).to(device)
#             g_optimizer.zero_grad()
#             gpred = gnet(X_batch)
#             gprediction.extend(gpred[:,1])
#             g_loss = loss_func(gpred,g_labels)
#             g_loss.sum().backward()
#             g_optimizer.step()
#             g_train_loss += float(g_loss.mean())

            
#         train_losses.append(g_train_loss/num_batches)
#         print('g_loss:',g_train_loss/num_batches) 
        
#         with torch.no_grad():
#             val_gloss = 0
#             for i in range(val_num_batches):
#                 val_X_batch = val_X[i * batch_size: (i + 1) * batch_size].to(device)
#                 val_Y_batch = val_Y[i * batch_size: (i + 1) * batch_size].to(device)
#                 val_hloss_batch = val_hloss[i * batch_size: (i + 1) * batch_size].to(device)
#                 val_machine_loss = loss_func(mnet(val_X_batch),val_Y_batch)
#                 val_machine_indices = find_machine_samples(val_machine_loss,val_hloss_batch,constraint=1.0)
#                 val_glabels = torch.tensor([0 if j in val_machine_indices else 1 for j in range(val_X_batch.shape[0])]).to(device)
#                 val_loss = loss_func(gnet(val_X_batch),val_glabels)
#                 val_gloss += float(val_loss.mean())

                
#             val_gloss /= val_num_batches
#             val_losses.append(val_gloss)
#             print('val_g_loss:',float(val_gloss))

#             if val_gloss + eps < best_val_loss:
#                 torch.save(gnet.state_dict(), model_dir + 'g_' + machine_type)
#                 best_val_loss = val_gloss
#                 print('updated the model')
#                 patience = 0
#             else:
#                 patience += 1

#         if patience > max_patience:
#             print('no progress for 10 epochs... stopping training')
#             break
                
#         print('\n')
        
#     del gnet
#     del mnet
        
#     plt.plot(range(len(train_losses)),train_losses,marker = 'o',label='train')
#     plt.plot(range(len(val_losses)),val_losses,marker = 'o',label='validation')
#     plt.title('train and validation curve of g' ,fontsize=22)
#     plt.xlabel('Epoch',fontsize=22)
#     plt.ylabel(r'g Loss',fontsize=20)
#     plt.legend()
#     plt.show()

In [ ]:
# train_g_full(data_path)

In [ ]:
# def get_test_assignments_full(constraints):
#     with torch.no_grad():
#         machine_type = 'full'
#         threshold_map = {0.0:0.0,0.2:0.30,0.4:0.30,0.6:0.30,0.8:0.21,1.0:0.37}
#         loss_func = torch.nn.NLLLoss(reduction='none')
#         data = load_data(data_path)
#         test_X = torch.from_numpy(data['test']['X']).float().to(device)
#         test_Y = data['test']['Y']
#         hlabel = data['test']['hpred']
#         hcrossloss = data['test']['hloss']

#         mnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=3, dropout=0.5).to(device)
#         mnet.to(device)
#         mnet.load_state_dict(torch.load(model_dir + 'm_full'))
#         mnet.eval()
#         mscores = mnet(test_X)
#         mcrossloss=loss_func(mscores,torch.tensor(test_Y).to(device))
#         mlabel = torch.argmax(mscores,dim=1).cpu().data.numpy()

#         gnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=2, dropout=0.5).to(device)
#         gnet.to(device)
#         gnet.load_state_dict(torch.load(model_dir + 'g_full'))
#         gnet.eval()
#         gprediction = torch.exp(gnet(test_X).detach()[:,1])
        
        
#         argsorted_g = torch.argsort(gprediction)
        

#         losses = []
#         for constraint in constraints:
#             num_machine = int((1.0 - constraint) * test_X.shape[0])
#             loss = np.zeros(test_X.shape[0])

#             human_candidates = torch.argsort(gprediction)[num_machine:]
#             to_machine = [i for i in range(mlabel.shape[0]) if i not in human_candidates or gprediction[i]<threshold_map[constraint]]
#             to_human = np.array([i for i in range(test_X.shape[0]) if i not in to_machine])
            
#             mloss = np.not_equal(mlabel,test_Y)
#             hloss = np.not_equal(hlabel,test_Y)
#             print(len(to_machine),len(to_human))
#             if len(to_machine)!=0:
#                 loss[to_machine] = mloss[to_machine]
#                 print('mean of machine error:' ,np.mean(loss[to_machine]))
#             if to_human.shape[0]!=0:
#                 loss[to_human] = hloss[to_human]
#                 print('mean of human error:' ,np.mean(loss[to_human]))
                
#             losses.append(np.mean(loss))
                

#     plt.plot(constraints,losses,marker='o')
#     plt.title(r'Full Automaion Triage',fontsize=22)
#     plt.ylabel(r'misclassification error',fontsize=22)
#     plt.xlabel(r'b',fontsize=22)
#     plt.show()
    
#     if machine_type not in data:
#         data[machine_type] = {}
        
#     data[machine_type]['agg_loss'] = losses
#     save_data(data,data_path)


In [ ]:
# get_test_assignments_full(constraints)

## [Score-based Triage](https://arxiv.org/abs/1903.12220)

In [ ]:
for seed in ['', 948,  625,  436,  791, 1750]:
  if seed != '':
    set_seed(seed)
  train_full(seed, data_path,'score')

In [ ]:
def get_assignments_score(data_path,constraints):
    machine_type = 'score'
    l = []
    for seed in ['', 948,  625,  436,  791, 1750]:
      if seed != '':
        mct = machine_type + '_seed_' + str(seed)
      else:
        mct = machine_type
      loss_func = torch.nn.NLLLoss(reduction='none')
      data = load_data(data_path)
      test_X = torch.from_numpy(data['test']['X']).float().to(device)
      test_Y = data['test']['Y']
      hlabel = data['test']['hpred']
      
      mnet = CNN(vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=3, dropout=0.5).to(device)

      mnet.load_state_dict(torch.load(model_dir + 'm_' + mct))
      mnet.eval()
      mlabel = torch.argmax(mnet(test_X),dim=1).cpu().data.numpy()
      loss = np.zeros(test_X.shape[0])
      mconf,_ = torch.max(mnet(test_X),axis = 1)
      
      losses = []
      for constraint in constraints:
          num_machine = int((1.0-constraint) * test_X.shape[0])
          to_machine = torch.argsort(mconf,descending = True)[:num_machine].cpu().data.numpy()
          to_human = np.array([i for i in range(test_X.shape[0]) if i not in to_machine])

          mloss = np.not_equal(mlabel,test_Y)
          hloss = np.not_equal(hlabel,test_Y)

          loss[to_machine] = mloss[to_machine]
          loss[to_human] =  hloss[to_human]

          losses.append(np.mean(loss))
      l.append(losses)
        
    if machine_type not in data.keys():
        data[machine_type] = {}

    l = np.array(l)
    print('l shape: ', l.shape)
    data[machine_type]['agg_loss'] = l
    save_data(data,data_path)


In [ ]:
get_assignments_score(data_path,constraints)

## [Surrogate-based Triage](https://arxiv.org/abs/2006.01862)
The code is taken from [here](https://github.com/clinicalml/learn-to-defer) where the authors provide implementation of their method.

In [ ]:
def metrics_print(net, val_X, val_Y, val_human_is_correct):
    net.eval()
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    alone_correct = 0
    batch_size = 64
    num_batches = int(val_X.shape[0] / batch_size)
    n_classes = 3
    with torch.no_grad():
        for i in range(num_batches):
            X_batch = val_X[i * batch_size:(i + 1) * batch_size]
            Y_batch = val_Y[i * batch_size:(i + 1) * batch_size]
            val_human_is_correct_batch = val_human_is_correct[i * batch_size:(i + 1) * batch_size]
            outputs = net(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            batch_size = outputs.size()[0]            # batch_size
            for i in range(0,batch_size):
                r = (predicted[i].item() == 3)
                prediction = predicted[i]
                if predicted[i] == n_classes:
                    max_idx = 0
					          # get second max
                    for j in range(0, n_classes):
                        if outputs.data[i][j] >= outputs.data[i][max_idx]:
                          max_idx = j
                    prediction = max_idx
                else:
                  prediction = predicted[i]
                alone_correct += (prediction == Y_batch[i]).item()
                if r==0:
                    total += 1
                    correct += (predicted[i] == Y_batch[i]).item()
                    correct_sys += (predicted[i] == Y_batch[i]).item()
                if r==1:
                    exp += val_human_is_correct_batch[i].item()
                    correct_sys += val_human_is_correct_batch[i].item()
                    exp_total+=1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print={"coverage":cov, "system accuracy": 100*correct_sys/real_total, "expert accuracy":100* exp/(exp_total+0.0002),"classifier accuracy":100*correct/(total+0.0001), "alone classifier": 100*alone_correct/real_total }
    print(to_print)
    return [100*total/real_total,  100*correct_sys/real_total, 100* exp/(exp_total+0.0002),100*correct/(total+0.0001) ]


def surrogate_train(seed, data_path, alpha=1.0):

    def loss_func(outputs, m, labels, m2, n_classes):
        '''
        The L_{CE} loss implementation for hatespeech, identical to CIFAR implementation
        ----
        outputs: network outputs
        m: cost of deferring to expert cost of classifier predicting (alpha* I_{m\neq y} + I_{m =y})
        labels: target
        m2:  cost of classifier predicting (alpha* I_{m\neq y} + I_{m =y})
        n_classes: number of classes
        '''
        batch_size = outputs.shape[0] # batch_size
        rc = [n_classes] * batch_size

        rc = torch.tensor(rc)

        outputs = -m * torch.log2(outputs[range(batch_size), rc]) - m2 * torch.log2(outputs[range(batch_size), labels])  # pick the values corresponding to the labels
        return torch.sum(outputs)/batch_size
    
    print('-----training machine model : surrogate')
    machine_type = 'surrogate'
    if seed != '':
      mct = machine_type + '_seed_' + str(seed)
    else:
      mct = machine_type
    data = load_data(data_path)
    X = torch.from_numpy(data['X']).float().to(device)
    Y = torch.from_numpy(data['Y']).to(device).long()
    human_is_correct = torch.from_numpy(np.array([1 if data['hpred'][i]==data['Y'][i] else 0
                                                  for i in range(X.shape[0])])).to(device)

    print(torch.mean(human_is_correct.float()))
    #alpha = 1.0
    m = (human_is_correct) * 1.0
    m2 = [0.0] * X.shape[0]
    for i, j in enumerate(m):
      if j == 1.0:
        m2[i] = alpha
      if j == 0.0:
        m2[i] = 1.0
    m2 = torch.tensor(m2)
    m2 = m2.to(device)


    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).to(device).long()
    val_human_is_correct = torch.from_numpy(np.array([1 if data['val']['hpred'][i] == data['val']['Y'][i] else 0
                                                      for i in range(val_X.shape[0])])).to(device)
    val_m = (val_human_is_correct) * 1.0
    val_m2 = [0.0] * val_X.shape[0]
    for i, j in enumerate(val_m):
      if j == 1.0:
        val_m2[i] = alpha
      if j == 0.0:
        val_m2[i] = 1.0
    val_m2 = torch.tensor(val_m2)
    val_m2 = val_m2.to(device)
    
    batch_size = 64
    num_epochs = 100

    num_batches = int(X.shape[0] / batch_size)
    
    N_FILTERS = 300  # hyperparameterr
    FILTER_SIZES = [3, 4, 5]
    DROPOUT = 0.5
    output_dim = 3
    mnet = CNN_rej(embedding_dim = 100,vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)
    print(mnet)
    optimizer = torch.optim.Adam(mnet.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_batches * num_epochs)
    train_losses = []
    val_losses = []
    best_val_loss = 1000
    max_patience = 10
    patience = 0
    eps = 1e-4

    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        epoch_loss = 0
        val_epoch_loss = 0
        for i in range(num_batches):
            X_batch = X[i * batch_size:(i + 1) * batch_size]
            Y_batch = Y[i * batch_size:(i + 1) * batch_size]
            m_batch = m[i * batch_size:(i + 1) * batch_size]
            m2_batch = m2[i * batch_size:(i + 1) * batch_size]

            optimizer.zero_grad()
            loss = loss_func(mnet(X_batch),m_batch,Y_batch,m2_batch,output_dim)
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)


        print('train loss: ',epoch_loss/ num_batches)
        train_losses.append(epoch_loss / num_batches)
        _ = metrics_print(mnet, val_X, val_Y, val_human_is_correct)
        with torch.no_grad():
            m_val_prob = mnet(val_X)
            val_loss = loss_func(m_val_prob, val_m, val_Y, val_m2, output_dim)
            val_losses.append(float(val_loss))
            print('validation loss: ', float(val_loss))
            
            if val_loss < best_val_loss:
                torch.save(mnet.state_dict(), model_dir + 'm_' + mct)
                best_val_loss = val_loss
                print('updated the model')
                patience = 0
            else:
                patience += 1
            
            if patience > max_patience:
                print('no progress for 10 epochs... stopping training')
                break
    
        print('\n')

    plt.plot(range(len(train_losses)),train_losses,marker='o', label = 'train')
    plt.plot(range(len(val_losses)),val_losses,marker='o', label = 'validation')
    plt.legend()
    plt.title(machine_type,fontsize=22)
    plt.xlabel(r'Time Step t',fontsize=22)
    plt.ylabel(r'Machine Loss',fontsize=20)
    plt.legend()
    plt.show()

In [ ]:
for seed in ['', 948,  625,  436,  791, 1750]:
    print("training with seed {}".format(seed))
    if seed != '':
      set_seed(seed)
    surrogate_train(seed, data_path)

In [ ]:
# data = load_data(data_path)
# test_X = torch.from_numpy(data['test']['X']).float().to(device)
# test_Y = data['test']['Y']
# hlabel = data['test']['hpred']
# test_human_is_correct = torch.from_numpy(np.array([1 if data['test']['hpred'][i] == data['test']['Y'][i] else 0
#                                                       for i in range(test_X.shape[0])])).to(device)
# mnet = CNN_rej(embedding_dim=100,vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=3, dropout=0.5).to(device)

# mnet.load_state_dict(torch.load(model_dir + 'm_surrogate')) # change accordingly for different seeds
# mnet.eval()

# metrics_print(mnet, test_X, test_Y, test_human_is_correct)

In [ ]:
def get_test_assignments_surrogate(data_path,constraints):
    machine_type = 'surrogate'
    l = []
    for seed in ['', 948,  625,  436,  791, 1750]:
      if seed != '':
        mct = machine_type + '_seed_' + str(seed)
      else:
        mct = machine_type
      loss_func = torch.nn.NLLLoss(reduction='none')
      data = load_data(data_path)
      test_X = torch.from_numpy(data['test']['X']).float().to(device)
      test_Y = data['test']['Y']
      hlabel = data['test']['hpred']
      
      mnet = CNN_rej(embedding_dim=100,vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=3, dropout=0.5).to(device)

      mnet.load_state_dict(torch.load(model_dir + 'm_' + mct))
      mnet.eval()
      mscores = mnet(test_X)
      temp = torch.sum(mscores[:,:3], dim=1, keepdim=True)
      mscores = torch.divide(mscores, temp)
      assert(mscores.shape[1]==4)
      mlabel = torch.argmax(mscores[:,:-1],dim=1).cpu().data.numpy()
      assert (mlabel<4).all()
      loss = np.zeros(test_X.shape[0])
      last_class_prob = mscores[:,-1]
      highest_prob,_ = torch.max(mscores[:,:-1],dim=1)
      diff =  last_class_prob - highest_prob
      
      hloss = np.not_equal(hlabel,test_Y)

      losses = []
      for constraint in constraints:
          num_machine = int((1.0-constraint) * test_X.shape[0])
          to_machine = torch.argsort(diff)[:num_machine].cpu().data.numpy()
          to_human = np.array([i for i in range(test_X.shape[0]) if i not in to_machine])

          mloss = np.not_equal(mlabel,test_Y)
          hloss = np.not_equal(hlabel,test_Y)

          loss[to_machine] = mloss[to_machine]
          loss[to_human] =  hloss[to_human]

          losses.append(np.mean(loss))
      
      l.append(losses)

    if machine_type not in data.keys():
        data[machine_type] = {}
    l = np.array(l)
    data[machine_type]['agg_loss'] = l
    save_data(data,data_path)


In [ ]:
get_test_assignments_surrogate(data_path,constraints)

### OvA Loss

In [ ]:
def metrics_print_ova(net, val_X, val_Y, val_human_is_correct):
    net.eval()
    correct = 0
    correct_sys = 0
    exp = 0
    exp_total = 0
    total = 0
    real_total = 0
    alone_correct = 0
    batch_size = 64
    num_batches = int(val_X.shape[0] / batch_size)
    n_classes = 3
    with torch.no_grad():
        for i in range(num_batches):
            X_batch = val_X[i * batch_size:(i + 1) * batch_size]
            Y_batch = val_Y[i * batch_size:(i + 1) * batch_size]
            val_human_is_correct_batch = val_human_is_correct[i * batch_size:(i + 1) * batch_size]
            _, outputs = net(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            batch_size = outputs.size()[0]            # batch_size
            for i in range(0,batch_size):
                r = (predicted[i].item() == 3)
                prediction = predicted[i]
                if predicted[i] == n_classes:
                    max_idx = 0
                              # get second max
                    for j in range(0, n_classes):
                        if outputs.data[i][j] >= outputs.data[i][max_idx]:
                          max_idx = j
                    prediction = max_idx
                else:
                  prediction = predicted[i]
                alone_correct += (prediction == Y_batch[i]).item()
                if r==0:
                    total += 1
                    correct += (predicted[i] == Y_batch[i]).item()
                    correct_sys += (predicted[i] == Y_batch[i]).item()
                if r==1:
                    exp += val_human_is_correct_batch[i].item()
                    correct_sys += val_human_is_correct_batch[i].item()
                    exp_total+=1
                real_total += 1
    cov = str(total) + str(" out of") + str(real_total)
    to_print={"coverage":cov, "system accuracy": 100*correct_sys/real_total, "expert accuracy":100* exp/(exp_total+0.0002),"classifier accuracy":100*correct/(total+0.0001), "alone classifier": 100*alone_correct/real_total }
    print(to_print)
    return [100*total/real_total,  100*correct_sys/real_total, 100* exp/(exp_total+0.0002),100*correct/(total+0.0001) ]

def loss_func_ova(outputs, labels, m, m2, n_classes):
    batch_size = outputs.size()[0]
    l1 = LogisticLoss(outputs[range(batch_size), labels], 1)
    l2 = torch.sum(LogisticLoss(outputs[:,:n_classes], -1), dim=1) - LogisticLoss(outputs[range(batch_size),labels],-1)
    l3 = LogisticLoss(outputs[range(batch_size), n_classes], -1)
    l4 = LogisticLoss(outputs[range(batch_size), n_classes], 1)

    l5 = m*(l4 - l3)

    l = m2*(l1 + l2) + l3 + l5
    return torch.mean(l)


def LogisticLoss(outputs, y):
    outputs[torch.where(outputs==0.0)] = (-1*y)*(-1*np.inf)
    l = torch.log2(1 + torch.exp((-1*y)*outputs))
    return l



def surrogate_train_ova(seed, data_path, alpha=1.0):    
    print('-----training machine model : surrogate')
    machine_type = 'surrogate_ova'
    if seed != '':
      mct = machine_type + '_seed_' + str(seed)
    else:
      mct = machine_type
    data = load_data(data_path)
    X = torch.from_numpy(data['X']).float().to(device)
    Y = torch.from_numpy(data['Y']).to(device).long()
    human_is_correct = torch.from_numpy(np.array([1 if data['hpred'][i]==data['Y'][i] else 0
                                                  for i in range(X.shape[0])])).to(device)

    print(torch.mean(human_is_correct.float()))
    alpha = 1.0
    m = (human_is_correct) * 1.0
    m2 = [0.0] * X.shape[0]
    for i, j in enumerate(m):
      if j == 1.0:
          m2[i] = alpha
      if j == 0.0:
          m2[i] = 1.0
    m2 = torch.tensor(m2).to(device)



    val_X = torch.from_numpy(data['val']['X']).float().to(device)
    val_Y = torch.from_numpy(data['val']['Y']).to(device).long()
    val_human_is_correct = torch.from_numpy(np.array([1 if data['val']['hpred'][i] == data['val']['Y'][i] else 0
                                                      for i in range(val_X.shape[0])])).to(device)
    val_m = (val_human_is_correct) * 1.0
    val_m2 = [0.0] * val_X.shape[0]
    for i, j in enumerate(val_m):
      if j == 1.0:
        val_m2 = alpha
      if j == 0.0:
        val_m2 = 1.0
    val_m2 = torch.tensor(val_m2).to(device)
    
    batch_size = 64
    num_epochs = 200

    num_batches = int(X.shape[0] / batch_size)
    
    N_FILTERS = 300  # hyperparameterr
    FILTER_SIZES = [3, 4, 5]
    DROPOUT = 0.5
    output_dim = 3
    mnet = CNN_rej_ova(embedding_dim = 100,vocab_size=X.shape[1], n_filters=300, filter_sizes=[3,4,5],
               output_dim=3, dropout=0.5).to(device)
    print(mnet)
    optimizer = torch.optim.Adam(mnet.parameters(),lr=0.005)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_batches * num_epochs)
    train_losses = []
    val_losses = []
    best_val_loss = 1000
    max_patience = 10
    patience = 0
    eps = 1e-4

    for epoch in range(num_epochs):
        print('----- epoch:',epoch, '-----')
        epoch_loss = 0
        val_epoch_loss = 0
        for i in range(num_batches):
            X_batch = X[i * batch_size:(i + 1) * batch_size]
            Y_batch = Y[i * batch_size:(i + 1) * batch_size]
            m_batch = m[i * batch_size:(i + 1) * batch_size]
            m2_batch = m2[i * batch_size:(i + 1) * batch_size]
            optimizer.zero_grad()
            loss = loss_func_ova(mnet(X_batch)[0],Y_batch, m_batch, m2_batch, output_dim)
            loss.backward()
            optimizer.step()
            epoch_loss += float(loss)


        print('train loss: ',epoch_loss/ num_batches)
        train_losses.append(epoch_loss / num_batches)
        _ = metrics_print_ova(mnet, val_X, val_Y, val_human_is_correct)
        with torch.no_grad():
            m_val_prob,_ = mnet(val_X)
            val_loss = loss_func_ova(m_val_prob, val_Y, val_m, val_m2, output_dim)
            val_losses.append(float(val_loss))
            print('validation loss: ', float(val_loss))
            
            if val_loss < best_val_loss:
                torch.save(mnet.state_dict(), model_dir + 'm_' + mct)
                best_val_loss = val_loss
                print('updated the model')
                patience = 0
            else:
                patience += 1
            
            if patience > max_patience:
                print('no progress for 10 epochs... stopping training')
                break
    
        print('\n')
               
    plt.plot(range(len(train_losses)),train_losses,marker='o', label = 'train')
    plt.plot(range(len(val_losses)),val_losses,marker='o', label = 'validation')
    plt.legend()
    plt.title(machine_type,fontsize=22)
    plt.xlabel(r'Time Step t',fontsize=22)
    plt.ylabel(r'Machine Loss',fontsize=20)
    plt.legend()
    plt.show()

In [ ]:
for seed in ['', 948,  625,  436,  791, 1750]:
    print("training with seed {}".format(seed))
    if seed != '':
      set_seed(seed)
    surrogate_train_ova(seed, data_path)

In [ ]:
# data = load_data(data_path)
# test_X = torch.from_numpy(data['test']['X']).float().to(device)
# test_Y = data['test']['Y']
# hlabel = data['test']['hpred']
# test_human_is_correct = torch.from_numpy(np.array([1 if data['test']['hpred'][i] == data['test']['Y'][i] else 0
#                                                       for i in range(test_X.shape[0])])).to(device)
# mnet = CNN_rej_ova(embedding_dim=100,vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
#                output_dim=3, dropout=0.5).to(device)

# mnet.load_state_dict(torch.load(model_dir + 'm_surrogate_ova_new')) # change accordingly for different seeds
# mnet.eval()

# metrics_print_ova(mnet, test_X, test_Y, test_human_is_correct)

In [ ]:
def get_test_assignments_surrogate_ova(data_path, constraints):
    machine_type = 'surrogate_ova'
    l = []
    for seed in ['', 948,  625,  436,  791, 1750]:
      if seed != '':
        mct = machine_type + '_seed_' + str(seed)
      else:
        mct = machine_type
      loss_func = torch.nn.NLLLoss(reduction='none')
      data = load_data(data_path)
      test_X = torch.from_numpy(data['test']['X']).float().to(device)
      test_Y = data['test']['Y']
      hlabel = data['test']['hpred']
      
      mnet = CNN_rej_ova(embedding_dim=100,vocab_size=test_X.shape[1], n_filters=300, filter_sizes=[3,4,5],
                output_dim=3, dropout=0.5).to(device)

      mnet.load_state_dict(torch.load(model_dir + 'm_' + mct))
      mnet.eval()
      _, mscores = mnet(test_X)
      assert(mscores.shape[1]==4)
      mlabel = torch.argmax(mscores[:,:-1],dim=1).cpu().data.numpy()
      assert (mlabel<4).all()
      loss = np.zeros(test_X.shape[0])
      last_class_prob = mscores[:,-1]
      highest_prob,_ = torch.max(mscores[:,:-1],dim=1)
      diff =  last_class_prob - highest_prob
      
      hloss = np.not_equal(hlabel,test_Y)

      to_defer = torch.where(diff > 0)[0]

      ids,_ = torch.sort(to_defer)

      losses = []
      for constraint in constraints:
          num_machine = int((1.0-constraint) * test_X.shape[0])
          human = torch.argsort(diff)[num_machine:].cpu().data.numpy()

          if len(human) >= len(ids):
              human_candidates = ids
          else:
              human_candidates = human

          to_machine = [i for i in range(mlabel.shape[0]) if i not in human_candidates]
      
          to_human = np.array([i for i in range(mlabel.shape[0]) if i not in to_machine])

          mloss = np.not_equal(mlabel,test_Y)
          hloss = np.not_equal(hlabel,test_Y)

          loss[to_machine] = mloss[to_machine]
          loss[to_human] =  hloss[to_human]

          losses.append(np.mean(loss))
      l.append(losses)
    
    if machine_type not in data.keys():
        data[machine_type] = {}
    l = np.array(l)
    data[machine_type]['agg_loss'] = l
    save_data(data,data_path)


In [ ]:
get_test_assignments_surrogate_ova(data_path, constraints)

## Plotting


In [ ]:
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rc('font', weight='bold')
plt.style.use('seaborn')

def plot_misclassification_loss(data_path,machine_types):
    fig, ax = plt.subplots(figsize=(4.5,4))
    data = load_data(data_path)
    colors = {'surrogate': 'blue', 'surrogate_ova' : 'green',  'confidence': 'purple', 'score' : 'firebrick', 'differentiable': 'orange'}
    for machine_type in machine_types:
        agg_loss = data[machine_type]['agg_loss']
        print(agg_loss.shape)
        if machine_type == "Differentiable":
          machine_type = "differentiable"
        mean = np.mean(agg_loss, axis=0)
        standard_error = stats.sem(agg_loss, axis=0)
        ax.plot(constraints,mean,marker='o',label=machine_type, color=colors[machine_type])
        ax.fill_between(constraints, mean-standard_error, mean+standard_error, alpha=0.3, color=colors[machine_type])
    ax.set_xticks(constraints)
    ax.set_ylabel(r'$misclassification$ ' + '$error$',fontsize=15)
    ax.set_xlabel(r'$budget$',fontsize=15)
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.legend()
    plt.tight_layout()
    return fig

In [ ]:
from scipy import stats

In [ ]:
machine_types =  ['surrogate', 'surrogate_ova', 'score','confidence','Differentiable']
fig = plot_misclassification_loss(data_path,machine_types)